In [2]:
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import Model, GRB
from scipy import stats

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
def gurobi_SSKP(sample_k, r, c, q):
    # input:
    # sample_k (weight matrix): 2d numpy array of shape (k, m), where m is the number of items, k is the number of samples
    # r (reward vector): 1d numpy array of shape (m, )
    # c (cost vector): scalar
    # q (budget): scalar
    # output: 
    # x (selection vector): 1d numpy array of shape (m, )
    k, m = sample_k.shape
    model = Model("SSKP")
    x = model.addVars(m, vtype=GRB.BINARY, name="x")
    z = model.addVars(k, lb=0, vtype=GRB.CONTINUOUS, name="z")

    model.setObjective(sum(r[i] * x[i] for i in range(m)) - c/k * sum(z[j] for j in range(k)), GRB.MAXIMIZE)

    for j in range(k):
        model.addConstr(z[j] >= sum(sample_k[j, i] * x[i] for i in range(m)) - q, f"budget_{j}")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        x_opt = np.array([x[i].X for i in range(m)])
        z_opt = np.array([z[j].X for j in range(k)])
        return x_opt
    else:
        print("No optimal solution found.")
        return None



In [4]:
def sample_func(distribution_name, **dist_params):
     if hasattr(np.random, distribution_name):
        # Access the distribution function dynamically
        dist_func = getattr(np.random, distribution_name)
        # Generate samples
        samples = dist_func(**dist_params)
        return samples
     else:
        raise ValueError(f"Unsupported distribution: {distribution_name}")

In [5]:
def majority_vote(sample_n, B, k, eval_func, *eval_args):
    x_count = {}
    for _ in range(B):
        # choose k samples from total n samples
        sample_k = sample_n[np.random.choice(sample_n.shape[0], k, replace=False)]
        x_k = tuple(eval_func(sample_k, *eval_args))
        x_count[x_k] = x_count.get(x_k, 0) + 1
    
    x_max = max(x_count, key=x_count.get)
    return x_max

In [6]:
def optimal_eval(sample, x, r, c, q):
     # sample: large number * m matrix
     expected_cost = 0
     for w in sample:
          expected_cost += max(sum(w[i] * x[i] for i in range(len(x)))-q, 0)
     expected_cost = expected_cost/len(sample)
     opt = sum(r[i] * x[i] for i in range(len(x))) - c * expected_cost
     return opt

In [7]:
# Parameters setup

m = 3 # Number of items
c = 5 # Cost 
q = 1 # Budget
r = np.random.uniform(3, 4, size=m) # Rewards

# Generate samples for evaluation of optimal SAA and majority vote
large_number_sample = 1000000 
a_ls = np.full(m, 2) #np.random.rand(m)+1 # Pareto distribution parameter
arrays_list = []
for i in range(m):
    arrays_list.append(sample_func('pareto', size=large_number_sample, a=a_ls[i]))
    larger_sample = np.vstack(arrays_list).T

sample_number = np.array([2**i for i in range(10, 17)])
number_of_iterations = 10 # Number of iterations for each sample size (use to take average)

In [ ]:
# SAA and majority vote comparison

SAA_list = []
majority_list = []
for n in sample_number:
    SAA_intermediate = []
    majority_intermediate = []
    for _ in range(number_of_iterations):
        arrays_list = []
        for i in range(m):
            arrays_list.append(sample_func('pareto', size=n, a=a_ls[i]))
            sample_n = np.vstack(arrays_list).T
        SAA = majority_vote(sample_n, 1, n, gurobi_SSKP, r,c,q)
        SAA_intermediate.append(SAA)

        majority = majority_vote(sample_n, 100, int(n/10), gurobi_SSKP, r,c,q)
        majority_intermediate.append(majority)
        
    SAA_list.append(SAA_intermediate)
    majority_list.append(majority_intermediate)

SAA_obj_list = []
majority_obj_list = []
for i in range(len(sample_number)):
    SAA_obj = 0
    majority_obj = 0
    for j in range(number_of_iterations):
        SAA_obj += optimal_eval(larger_sample, SAA_list[i][j], r, c, q)
        SAA_obj = SAA_obj/number_of_iterations
        majority_obj += optimal_eval(larger_sample, majority_list[i][j], r, c, q)
        majority_obj = majority_obj/number_of_iterations

    SAA_obj_list.append(SAA_obj)
    majority_obj_list.append(majority_obj)
        

fig2, ax2 = plt.subplots()
ax2.plot(sample_number, SAA_obj_list, marker = 'o', markeredgecolor = 'none', color = 'blue',linestyle = 'solid', linewidth = 2, label = 'SAA')
ax2.plot(sample_number, majority_obj_list, marker = 's', markeredgecolor = 'none', color = 'red',linestyle = 'solid', linewidth = 2, label = 'Majority Vote')
ax2.set_xlabel('Number of samples', size = 20)
ax2.set_ylabel('Objective', size = 20)
ax2.legend(loc = 'lower right')
plt.show()

In [ ]:
start = 3
end = 7
fig2, ax2 = plt.subplots()
ax2.plot(sample_number[start:end], SAA_obj_list[start:end], marker = 'o', markeredgecolor = 'none', color = 'blue',linestyle = 'solid', linewidth = 2, label = 'SAA')
ax2.plot(sample_number[start:end], majority_obj_list[start:end], marker = 's', markeredgecolor = 'none', color = 'red',linestyle = 'solid', linewidth = 2, label = 'Majority Vote')
ax2.set_xlabel('Number of samples', size = 20)
ax2.set_ylabel('Objective', size = 20)
ax2.legend(loc = 'lower right')
plt.show()

In [8]:
# Testing: SAA and majority vote comparison - parameters setup

sample_number = np.array([2**i for i in range(10, 15)])
number_of_iterations = 5 # Number of iterations for each sample size (use to take average)
m = 3 # Number of items


# Generate samples for evaluation of optimal SAA and majority vote
large_number_sample = 100000
a_ls = np.full(m, 2) #np.random.rand(m)+1 # Pareto distribution parameter
arrays_list = []
for i in range(m):
    arrays_list.append(sample_func('pareto', size=large_number_sample, a=a_ls[i]))
    larger_sample = np.vstack(arrays_list).T

parameter_list = []
SAA_list_test = []
majority_list_test = []
SAA_obj_list_test = []
majority_obj_list_test = []

for num_test in range(5):
    print(num_test)
    c = float(np.random.uniform(3, 4, size=1)[0]) # Cost 
    q = float(np.random.uniform(0, 2, size=1)[0]) # Budget
    r = np.random.uniform(3, 4, size=m) # Rewards
    
    SAA_obj = 0
    majority_obj = 0
    flag = 0
    for n in sample_number:
        SAA_intermediate = []
        majority_intermediate = []
        for j in range(number_of_iterations):
            arrays_list = []
            for i in range(m):
                arrays_list.append(sample_func('pareto', size=n, a=a_ls[i]))
                sample_n = np.vstack(arrays_list).T
            SAA = majority_vote(sample_n, 1, n, gurobi_SSKP, r,c,q)
            SAA_intermediate.append(SAA)
            SAA_obj += optimal_eval(larger_sample, SAA, r, c, q)

            majority = majority_vote(sample_n, 300, int(n/10), gurobi_SSKP, r,c,q)
            majority_intermediate.append(majority)
            majority_obj += optimal_eval(larger_sample, majority, r, c, q)

        SAA_obj = SAA_obj/number_of_iterations
        majority_obj = majority_obj/number_of_iterations

        if SAA_obj > majority_obj:
            flag = 1
            break

    if flag == 1:
        continue 
    
    parameter_list.append([m, c, q, r])
    SAA_obj_list_test.append(SAA_obj)
    majority_obj_list_test.append(majority_obj)

0
Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-25
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 21.3.0 21D62)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1024 rows, 1027 columns and 4096 nonzeros
Model fingerprint: 0xbea43645
Variable types: 1024 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e-04, 7e+01]
  Objective range  [3e-03, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 4.6195818
Presolve removed 1024 rows and 1027 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 3: 4.61958 4.61958 -0 
No other solutions better than 4.61958

Optimal soluti

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Optimize a model with 102 rows, 105 columns and 408 nonzeros
Model fingerprint: 0xbbcdc5a4
Variable types: 102 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e-03, 3e+01]
  Objective range  [3e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 3.0114716
Presolve removed 102 rows and 105 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 3: 3.01147 3.01147 -0 
No other solutions better than 3.01147

Optimal solution found (tolerance 1.00e-04)
Best objective 3.011471606018e+00, best bound 3.011471606018e+00, gap 0.0000%
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 21.3.0 21D62)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Solution count 3: 4.94076 4.94076 -0 
No other solutions better than 4.94076

Optimal solution found (tolerance 1.00e-04)
Best objective 4.940760610619e+00, best bound 4.940760610619e+00, gap 0.0000%
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 21.3.0 21D62)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 102 rows, 105 columns and 408 nonzeros
Model fingerprint: 0x4444058f
Variable types: 102 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [2e-03, 7e+01]
  Objective range  [3e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 1.5515696
Presolve removed 69 rows and 69 columns
Presolve time: 0.00s
Presolved: 33 rows, 36 columns, 100 nonzeros
Found heuristic solution: objective 1.5515695
Found heuristic solution: objective 2.6760229
Variable types: 34

TypeError: 'NoneType' object is not iterable

In [1]:
len(parameter_list)

NameError: name 'parameter_list' is not defined

In [ ]:
print(len(SAA_obj_list_test))
print(SAA_obj_list_test)
print(majority_obj_list_test)

In [ ]:
# normal distribution comparison between SAA and majority vote
def u(x, u, q):
    return sum(u_i * x_i for u_i, x_i in zip(u, x)) - q

def var(x,std):
    return sum(std_i**2 * x_i**2 for std_i, x_i in zip(std, x))

def phi(x):
    return stats.norm.cdf(x, loc=0, scale=1)

def g(x, r, c, q, avg, std):
    expect_obj = sum(r_i * x_i for r_i, x_i in zip(r, x)) \
        - c * (u(x,avg,q) * phi(u(x,avg,q)/np.sqrt(var(x,std))) \
               + np.sqrt(var(x,std)/(2*np.pi)) * np.exp(-u(x,avg,q)**2/(2*var(x,std))))
    return expect_obj

m = 3 # Number of items
c = 5 # Cost 
q = 10 # Budget
r = np.random.uniform(3, 4, size=m) # Rewards

# generate n samples with iid normal distribution
avg =  np.random.randint(1, 5, size=m)
std =  np.random.randint(10, 20, size=m)


SAA_list_normal = []
SAA_obj_list_normal = []
majority_list_normal = []
majority_obj_list_normal = []
sample_number = np.arange(20, 1050, 50)
for ite in sample_number:
    arrays_list = []
    n = ite
    for i in range(m):
        arrays_list.append(sample_func('normal', size=n, loc=avg[i], scale=std[i]))
        sample_n = np.vstack(arrays_list).T
    SAA = majority_vote(sample_n, 1, n, gurobi_SSKP, r,c,q)
    SAA_list_normal.append(SAA)
    SAA_obj = g(SAA, r, c, q, avg, std)
    SAA_obj_list_normal.append(SAA_obj)

    majority = majority_vote(sample_n, 40, int(n/10), gurobi_SSKP, r,c,q)
    majority_list_normal.append(majority)
    majority_obj = g(majority, r, c, q, avg, std)
    majority_obj_list_normal.append(majority_obj)

fig1, ax1 = plt.subplots()
ax1.plot(sample_number, SAA_obj_list_normal, marker = 'o', markeredgecolor = 'none', color = 'blue',linestyle = 'solid', linewidth = 2, label = 'SAA')
ax1.plot(sample_number, majority_obj_list_normal, marker = 's', markeredgecolor = 'none', color = 'red',linestyle = 'solid', linewidth = 2, label = 'Majority Vote')
ax1.set_xlabel('Number of samples', size = 20)
ax1.set_ylabel('Objective', size = 20)
ax1.legend(loc = 'lower right')
plt.show()

In [ ]:
print(SAA_list_normal)
print(majority_list_normal)
print(SAA_obj_list_normal)
print(majority_obj_list_normal)